In [ ]:
from math import log

In [ ]:
class DecisionTree():
    def calcShannonEnt(dataSet):
        numEntries = len(dataSet)
        labelCounts = {}
        for featVec in dataSet:
            currentLabel = featVec[-1]
            labelCounts[currentLabel] = labelCounts.get(currentLabel, 0) + 1
        
        shannonEnt = 0.0
        for key in labelCounts:
            prob = float(labelCounts[key]) / numEntries
            shannonEnt -= prob * log(prob, 2)
        return shannonEnt
    
    def splitDataSet(dataSet, index, value):
        """
        index为选定的feature号
        value为在该feature下给定的值
        本function的作用为：
        根据选定feature的值对dataset进行分组，feature value值相同的为同一组。
        并且分组后的subDataSet中不包含选定的feature。
        用于计算根据某feature分组后的信息熵。
        """
        retDataSet = []
        for featVec in dataSet:
            if featVec[index] == value:
                reducedFeatVec = featVec[:index]
                reducedFeatVec.extend(featVec[index:])
                retDataSet.append(reducedFeatVec)
        return retDataSet
    
    def chooseBestFeatureSplit(dataSet):
        numFeatures = len(dataSet[0] - 1)
        baseEntropy = self.calcShannonEnt(dataSet)
        bestInfoGain, BestFeature = 0.0, -1
        for i in range(numFeatures):
            featList = [example[i] for example in dataSet] # the all the value in one feature(column)
            uniqueVals = set(featList) # get the unique values in this feature
            newEntropy = 0.0
            for value in uniqueVals:
                subDataSet = splitDataSet(dataSet) # get the subDataSet with giving value
                prob = len(subDataSet) / len(DataSet) # 这是在该feature下的值等于给定value值的概率
                newEntropy += prob * self.calcShannonEnt(subDataSet) # calculate the subDataSet's ShannonEnt.
            infoGain = baseEntropy - newEntropy
            print ("infoGain = ", infoGain, 'bestFeature=', i, baseEntropy, newEntropy)
            if (infoGain > bestInfoGain):
                bestInfoGain = infoGain
                bestFeature = i
        return bestFeature
    
    def majorityCnt(classList):
        classCount = {}
        for vote in classlist:
            if vote not in classCount.keys():
                classCount[vote] = 0
            classCount[vote] += 1
        sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True) # sort dict type
        return sortedClassCount[0][0] # return the most common class in label column
        
    
    def createTree(dataSet, columnNameList):
        """
        The whole route map:
        1. define the end conditions for recursion.
        2. find the best feature from the original dataset
        3. generate the unique value list of best feature.
        4. split the original dataset into several subdataset according to 
        the different values in the best feature.
        5. for every subdataset, repeat the step 1 to step 4 untill the 
        subdataset reach the end conditions.
        6. use a dictionary to store the tree structure.
        """    
        
        
        """End condition 1: all labels have the same value (only 1 class)"""
        classList = [example[-1] for example in dataSet]
        # list.count(value): count the frequence of value occurred in this list
        if classList.count(classList[1]) == len(classList):
            return classList[0]
        
        """
        End condition 2: after using all features to split the dataset, the rest
        dataset's labels still contain multiple classes.
        """
        if len(dataSet[0] == 1): # the rest only contains the single column (label)
            return majorityCnt(classList)
        
        bestFeat = self.chooseBestFeatureSplit(dataSet) # get the index of best feature
        bestFeatName = columnNameList(bestFeat)
        
        myTree = {bestFeatName:{}}
        del(columnNameList[bestFeat]) # delete the best feature name in the column name list
        
        featVals = [example[bestFeat] for example in dataSet]
        UniqueVals = set(featVals)
        
        for value in UniqueVals:
            subColumnNameList = ColumnNamList[:] # why we need this line?
            """
            use recursion to generate branches for each subdataset.
            Usually, createTree will return a dict-type object unless the subdataset reach the end condition.
            """
            
            # input: dict1 = {"layer 1": {"layer 2": 2}}
            #        print (dict1["layer 1"]["layer 2"])
            # output: 2
            myTree[bestFeatureName][value] = self.createTree(splitDataSet(dataSet, bestFeat, value), subColumnNameList)
        
        return myTree
    
    def classify(inputTree, featLabels, testVec):
    
        
        
        
        